# Homework 2, Problem 5

In this problem we look at weather and how it impacts trading on the New York stock enchange. Complete this notebook and keep it in a homework 2 repo. Submit the repo link though the blackboard.

**You are free to add implmentation or markdown cells to make your notebook clearer!!**

## Data:

The following two datasets are our focus

* Weather data [NOAA-GHCN](https://registry.opendata.aws/noaa-ghcn/)
* Stock Exchange Data [Yahoo Finance](https://finance.yahoo.com/quote/%5ENYA/history?ltr=1) 



## Part 1: Download The Weather Data




Download a year of weather data.

The Raw GHCN files don't have column headers, so we manually add them in. It's safer to at this point read in everything as an object & then parse to the correct type once you extract the variables you're interested in. 
This information can be found in https://docs.opendata.aws/noaa-ghcn-pds/readme.html

In [1]:
import urllib 

import pandas as pd

import dask.dataframe as dd
import dask.bag as db
import dask.diagnostics as dg

We're using Dask for the lazy evaluation properties (it will only try to run the computations at the end, hopefully after the data has been filtered down) because the dataset is very large. We set the storage options to `anon=True` because this data is public. Otherwise this kwarg is where we'd pass in the AWS authorization keys. 

In [2]:
# Let's load in the data for 1992
YEAR = 1992

names = ['ID', 'DATE', 'ELEMENT', 'DATA_VALUE', 'M-FLAG', 'Q-FLAG', 'S-FLAG', 'OBS-TIME']
ds = dd.read_csv(f's3://noaa-ghcn-pds/csv/{YEAR}.csv', storage_options={'anon':True},  names=names, memory_map=False, 
                  dtype={'DATA_VALUE':'object'}, parse_dates=['DATE', 'OBS-TIME'])

In [3]:
# You can check the data
print(ds.columns)
print(ds.dtypes)

Index(['ID', 'DATE', 'ELEMENT', 'DATA_VALUE', 'M-FLAG', 'Q-FLAG', 'S-FLAG',
       'OBS-TIME'],
      dtype='object')
ID                    object
DATE          datetime64[ns]
ELEMENT               object
DATA_VALUE            object
M-FLAG                object
Q-FLAG                object
S-FLAG                object
OBS-TIME              object
dtype: object


In [4]:
# Print out the first few rows
ds.head()

,ID,DATE,ELEMENT,DATA_VALUE,M-FLAG,Q-FLAG,S-FLAG,OBS-TIME
0,CA002303986,1992-01-01,TMAX,-70,NaN,NaN,C,nan
1,CA002303986,1992-01-01,TMIN,-240,NaN,NaN,C,nan
2,CA002303986,1992-01-01,PRCP,4,NaN,NaN,C,nan
3,CA002303986,1992-01-01,SNOW,4,NaN,NaN,C,nan
4,CA002303986,1992-01-01,SNWD,420,NaN,NaN,C,nan


Now we want to parse out the station ID list. We are using [pandas.read_fwf](https://pandas.pydata.org/pandas-docs/version/0.23.4/generated/pandas.read_fwf.html#pandas.read_fwf) because this file is a fixed format width table rather than a csv file. 
We explicitly pass in the extents of the fixed width field because Pandas has trouble inferring what belongs in the `STATE` column versus in the `NAME` column. We obtained these extents from the readme https://docs.opendata.aws/noaa-ghcn-pds/readme.html

In [5]:
# {column name:extents of the fixed-width fields}
columns = {"ID": (0,11), "LATITUDE": (12, 20), "LONGITUDE": (21, 30), "ELEVATION": (31, 37),"STATE": (38, 40),
           "NAME": (41, 71), "GSN FLAG": (72, 75), "HCN/CRN FLAG": (76, 79),"WMO ID": (80, 85)}

In [6]:
df = pd.read_fwf("http://noaa-ghcn-pds.s3.amazonaws.com/ghcnd-stations.txt", 
                    colspecs=list(columns.values()), names=list(columns.keys()))

In [7]:
df.head()

,ID,LATITUDE,LONGITUDE,ELEVATION,STATE,NAME,GSN FLAG,HCN/CRN FLAG,WMO ID
0,ACW00011604,17.1167,-61.7833,10.1,NaN,ST JOHNS COOLIDGE FLD,NaN,NaN,NaN
1,ACW00011647,17.1333,-61.7833,19.2,NaN,ST JOHNS,NaN,NaN,NaN
2,AE000041196,25.3330,55.5170,34.0,NaN,SHARJAH INTER. AIRP,GSN,NaN,41196.0
3,AEM00041194,25.2550,55.3640,10.4,NaN,DUBAI INTL,NaN,NaN,41194.0
4,AEM00041217,24.4330,54.6510,26.8,NaN,ABU DHABI INTL,NaN,NaN,41217.0


In [8]:
# You should be looking for those in the New York area like Central Park, JFK, LGA and Newark airport.
NYNJ = df[df['STATE'].isin(['NY', 'NJ'])]
NYNJ.head()

,ID,LATITUDE,LONGITUDE,ELEVATION,STATE,NAME,GSN FLAG,HCN/CRN FLAG,WMO ID
73674,US1NJAT0001,39.5483,-74.8671,31.4,NJ,BUENA VISTA TWP 2.6 NNE,NaN,NaN,NaN
73675,US1NJAT0002,39.5565,-74.8048,14.0,NJ,FOLSOM 3.2 SE,NaN,NaN,NaN
73676,US1NJAT0003,39.4747,-74.7107,5.5,NJ,HAMILTON TWP 2.1 SE,NaN,NaN,NaN
73677,US1NJAT0005,39.6404,-74.8261,29.9,NJ,HAMMONTON 3.3 WSW,NaN,NaN,NaN
73678,US1NJAT0009,39.3346,-74.5759,5.8,NJ,LINWOOD 0.7 SSW,NaN,NaN,NaN


Central Park is coded in shorthand, so we used the NOAA web portal to look up the correct ID
https://www.ncdc.noaa.gov/cdo-web/datasets/GHCND/stations/GHCND:USW00094728/detail

In [9]:
NYNJ[NYNJ['ID'].str.contains('USW00094728')]

,ID,LATITUDE,LONGITUDE,ELEVATION,STATE,NAME,GSN FLAG,HCN/CRN FLAG,WMO ID
114226,USW00094728,40.7789,-73.9692,39.6,NY,NEW YORK CNTRL PK TWR,NaN,HCN,72506.0


In [10]:
# Airports + Central Park
apcp = NYNJ[NYNJ['NAME'].str.endswith('AP') | NYNJ['ID'].str.contains('USW00094728')]
apcp.head()

,ID,LATITUDE,LONGITUDE,ELEVATION,STATE,NAME,GSN FLAG,HCN/CRN FLAG,WMO ID
100219,USC00305840,43.1139,-78.9353,179.2,NY,NIAGARA FALLS INTL AP,NaN,NaN,NaN
112764,USW00004724,43.1072,-78.9453,178.3,NY,NIAGARA FALLS INTL AP,NaN,NaN,NaN
112769,USW00004742,44.6500,-73.4667,71.9,NY,PLATTSBURGH INTL AP,NaN,NaN,NaN
112775,USW00004781,40.7939,-73.1017,25.6,NY,ISLIP LI MACARTHUR AP,NaN,NaN,72505.0
112779,USW00004789,41.5092,-74.2650,111.3,NY,MONTGOMERY ORANGE AP,NaN,NaN,NaN


What we're interested in is the IDs, which we will use for our dataset to obtain only the stations of interest. We are going to join our two dataframes on the ID column so that we have all the information in every row.  We are removing the flags since they have neither computational nor necessary identification information. 

we do not use `.compute()` to resolve the computation because it's better to hold off until the completetion of feature selection and engineering described below. If you'd like a fully computed dataframe, the code is 
```python


In [11]:
nyds = ds.merge(apcp[['ID', 'LATITUDE', 'LONGITUDE', 'ELEVATION', 'STATE', 'NAME']], on='ID').compute()


In [12]:
nyds.head()

,ID,DATE,ELEMENT,DATA_VALUE,M-FLAG,Q-FLAG,S-FLAG,OBS-TIME,LATITUDE,LONGITUDE,ELEVATION,STATE,NAME
0,USW00094790,1992-01-01,TMAX,61,NaN,NaN,0,2400,43.9922,-76.0217,96.9,NY,WATERTOWN INTL AP
1,USW00094790,1992-01-01,TMIN,-133,NaN,NaN,0,2400,43.9922,-76.0217,96.9,NY,WATERTOWN INTL AP
2,USW00094790,1992-01-01,PRCP,0,NaN,NaN,0,2400,43.9922,-76.0217,96.9,NY,WATERTOWN INTL AP
3,USW00094790,1992-01-01,SNOW,0,NaN,NaN,0,nan,43.9922,-76.0217,96.9,NY,WATERTOWN INTL AP
4,USW00094790,1992-01-01,SNWD,0,NaN,NaN,0,nan,43.9922,-76.0217,96.9,NY,WATERTOWN INTL AP


## Part 2: Downoad Stock Price Data

Here the idea is to get the finance data from Yahoo finace.  It's already the right date range in general:

In [18]:
finance_df = pd.read_csv("https://query1.finance.yahoo.com/v7/finance/download/%5ENYA?period1=694224000&period2=725760000&interval=1d&events=history")
finance_df = finance_df.rename(columns={"Date": "DATE"})

In [19]:
finance_df.head()

,DATE,Open,High,Low,Close,Adj Close,Volume
0,1992-01-02,2423.179932,2423.179932,2423.179932,2423.179932,2423.179932,0
1,1992-01-03,2435.659912,2435.659912,2435.659912,2435.659912,2435.659912,0
2,1992-01-06,2430.370117,2430.370117,2430.370117,2430.370117,2430.370117,0
3,1992-01-07,2428.679932,2428.679932,2428.679932,2428.679932,2428.679932,0
4,1992-01-08,2434.389893,2434.389893,2434.389893,2434.389893,2434.389893,0


You can do an inner join for the dates from the financial dataset and the new york weather dataset, to get all your features ready, please do that here:

In [ ]:
# your join on dates goes here:
ny_df = nyds.compute()
ny_df["DATE"] = pd.to_datetime(ny_df["DATE"])
finance_df["DATE"] = pd.to_datetime(finance_df["DATE"])

## Part 3: Creating and Selecting Variables

Pull out and encode the various variables listed below and set up these varaibles at least initially in a pandas data frame.

### Weather variables

* raining:
    - 0 - wasn't raining
    - 1 - was raining
* rain intensity:
    - 0 -low
    - 1 - medium
    - 2 - high
* rain duration in hours
* snowing:
    - 0 - wasn't snowing
    - 1 - was snowing
* snow intensity:
    - 0 - low
    1 - medium
    2 - high
* snow duration in hours
* windy:
    - 0 - low
    - 1 - medium
    - 2 - high

### Market Variables 

* Market Open
* Market Close
* Market High
* Market Low
* Market Volume


Make sure you have aligned the data by date in a pandas data frame. Show the counts and the summary stats.

## Part 4: Feature Engineering

Because we are going to be thinking of this in terms of a simple neural network here (like a dense neural network), extend the data by the input data actually being the past $n$ days ($n$ between 1 and 7). In other words the $X$ input should contain a lag of variables you loaded, but lagged by days from 1 through 7. In other words if it hasn't snowed in the past 7 days you will have attributes $[0,0,0,0,0,0,0]$ for yesterday and the preceeding 8 days of no snow, being "columns" or dimensions in your input data.

One challenge is that for weekend you will not have trading days so you will need to do some data filling. After you "fatten" your data, should see if you need all this data. You should normalize all your input variables so that that have an approximate range between 0 and 1. 

In [39]:
from functools import partial

def lag_function(num_days, x):
    # fill in the code for the lag day here
    return x

lag_one_day = partial(lag_function, 1)
joined_data["market_volatility_lag_one"] = joined_data.apply(lag_one_day, axis=1)

lag_two_days = partial(lag_function, 2)
joined_data["market_volatility_lag_two"] = joined_data.apply(lag_two_days, axis=1)

lag_three_days = partial(lag_function, 3)
joined_data["market_volatility_lag_three"] = joined_data.apply(lag_three_days, axis=1)

lag_four_days = partial(lag_function, 4)
joined_data["market_volatility_lag_four"] = joined_data.apply(lag_four_days, axis=1)

lag_five_days = partial(lag_function, 5)
joined_data["market_volatility_lag_five"] = joined_data.apply(lag_five_days, axis=1)

lag_six_days = partial(lag_function, 6)
joined_data["market_volatility_lag_six"] = joined_data.apply(lag_six_days, axis=1)

lag_seven_days = partial(lag_function, 7)
joined_data["market_volatility_lag_seven"] = joined_data.apply(lag_seven_days, axis=1)


In [ ]:
# add the code for dealing with weekends here

In [ ]:
# add the code for normalization here

## Part 5: Try out different Models and prediction!

Your goal is to predict the volatility in the market, that is the Market High - Market Low your "Y" value. For convenience create that column. All of the other columns will help create your "X" variables. You can use any of the other variables as predictors. Be careful not use Market High or Market Low as "X" variables!

Since we are doing a regression problem that means that the last neural net activation will probably be linear and the loss should be Mean Squared Error or root mean squared error or mean absolute error.

Try five different models. For each model, please report mse, root mse and mean absolute error.  You can get the training history with:

Record the history with:

`history = model.fit(X, y, validation_split=0.1)`

and get the history for your metrics with:

`history.history`

For more details, see this tutorial: https://machinelearningmastery.com/custom-metrics-deep-learning-keras-python/

Also, please note the above tutorial shows you how to include multiple metrics with keras.

Then try cross validation with the above metrics. 

If you've never done cross validation with keras before, please use: https://machinelearningmastery.com/evaluate-performance-deep-learning-models-keras/

The above tutorial will show you how.

After running cross validation for each of the metrics you should be able to answer the following questions:

Is there overfitting? How do you know?
Why do you think certain models worked well and others not as well? 
How might you improve the model?

In [ ]:
# add your model architectures here

def model_one():
    pass

def model_two():
    pass

def model_three():
    pass

def model_four():
    pass

def model_five():
    pass

# evaluate your architectures on a test set here

In [ ]:
# implement cross validation here

answer answer overfitting here and analysis here.

## Overall Conclusion

Conclude with a full report here on what we know now about this problem. How well it does verses baseline, what the best Keras archtecture is, what features should be used, how the data should be cleaned etc.